<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/BB/bb_t5v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import json

# Make longer output readable without scrolling
from pprint import pprint

# Stop warning messages from showing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 5.6 MB/s 


In [3]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 5.4 MB/s 
     |████████████████████████████████| 163 kB 21.6 MB/s 
     |████████████████████████████████| 7.6 MB 44.6 MB/s 


In [4]:
!pip install -q datasets

     |████████████████████████████████| 441 kB 5.4 MB/s 
     |████████████████████████████████| 212 kB 40.6 MB/s 
     |████████████████████████████████| 115 kB 7.8 MB/s 
     |████████████████████████████████| 127 kB 44.8 MB/s 


In [5]:
!pip install -q evaluate
import evaluate

     |████████████████████████████████| 72 kB 773 kB/s 


In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datasets import list_datasets, load_dataset_builder, get_dataset_config_names, load_dataset, load_from_disk


In [8]:
def summarize_dataset (dataset, config=None):
    builder = load_dataset_builder(dataset, config)
    print(f"Description:\n {builder.info.description}")
    print(f"Features:")
    pprint(builder.info.features)
    return

### Load SQuAD dataset from Gdrive

In [9]:
data_squad = load_from_disk("/content/drive/MyDrive/w266 NLP Final Project/Data/squad.hf")

In [10]:
data_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [11]:
# Look at first example
pprint(data_squad['train'][0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

### T5v1.1 Model

Sources: https://huggingface.co/docs/transformers/model_doc/t5
https://huggingface.co/docs/transformers/model_doc/t5v1.1

In [12]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

model = TFT5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")


# model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/991M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-v1_1-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [13]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  84954240  
                                                                 
 decoder (TFT5MainLayer)     multiple                  113275008 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247,577,856
Trainable params: 247,577,856
Non-trainable params: 0
_________________________________________________________________


### Input Structure

The input data should be a single list of dictionaries (or path to a JSON file containing the same). A dictionary represents a single context and its associated questions.

Each such dictionary contains two attributes, the "context" and "qas".

context: The paragraph or text from which the question is asked.
qas: A list of questions and answers (format below).
Questions and answers are represented as dictionaries. Each dictionary in qas has the following format.

id: (string) A unique ID for the question. Should be unique across the entire dataset.

question: (string) A question.

answers: (list) The list of correct answers to the question.

Source: https://simpletransformers.ai/docs/qa-data-formats/

In [33]:
# Shuffle the dataset and take a handful of examples

count=25
sample=data_squad['train'].shuffle(seed=1962).select(range(count))
df=pd.DataFrame()
df['answer'] = [answer['text'][0] for answer in sample['answers']]
df['context'] = sample['context']
df['question'] = sample['question']

df

,answer,context,question
0,biotech companies,"Prior to moving its headquarters to Chicago, a...",What type of businesses did Nickles want to at...
1,Tytus Woyciechowski,Four boarders at his parents' apartments becam...,To whom did Chopin reveal in letters which par...
2,the Endangered Species Committee,The question to be answered is whether a liste...,"If a species may be harmed, who holds final sa..."
3,China,"In Asian countries such as China, Korea, and J...",What country has the dog as part of its 12 ani...
4,45 years,Saint Athanasius of Alexandria (/ˌæθəˈneɪʃəs/;...,How long did his episcopate last?
5,"Cold War, First Gulf War, Kosovo War","Since 1947, Canadian military units have parti...",What are some of the wars the Canadian Militar...
6,Buddha,Tibet has various festivals that are commonly ...,What is worshipped during Tibet's various fest...
7,9.3%,"From 2001 to 2008, Mac sales increased continu...",What was Apples market share of all computer s...
8,Improvisation,Improvisation stands at the centre of Chopin's...,What is central to Chopin's process?
9,1861,"Alfred North Whitehead was born in Ramsgate, K...",What year was Whitehead born?


## Try training the model

Source: https://huggingface.co/docs/transformers/model_doc/t5#transformers.TFT5ForConditionalGeneration

Prep data for inputs and outputs

In [34]:
# Come up with a prompt for T5 model
prompt = 'question_gen: '
# Add "answer: " to each answer in list
# answer = ["question_gen: answer: " + a for a in df.answer]
# Add "context: " to each context in list
context = ["context: " + c for c in df.context]
# Add "question: " to each question in list
question = ["question: " + q for q in df.question]

# Input text will be prompt + answer + context
input_txt = [prompt + x for x in context]

# Output text will be questions
output_txt = question

input_txt

["question_gen: context: Prior to moving its headquarters to Chicago, aerospace manufacturer Boeing (#30) was the largest company based in Seattle. Its largest division is still headquartered in nearby Renton, and the company has large aircraft manufacturing plants in Everett and Renton, so it remains the largest private employer in the Seattle metropolitan area. Former Seattle Mayor Greg Nickels announced a desire to spark a new economic boom driven by the biotechnology industry in 2006. Major redevelopment of the South Lake Union neighborhood is underway, in an effort to attract new and established biotech companies to the city, joining biotech companies Corixa (acquired by GlaxoSmithKline), Immunex (now part of Amgen), Trubion, and ZymoGenetics. Vulcan Inc., the holding company of billionaire Paul Allen, is behind most of the development projects in the region. While some see the new development as an economic boon, others have criticized Nickels and the Seattle City Council for pan

In [35]:
output_txt

['question: What type of businesses did Nickles want to attract to Seattle?',
 'question: To whom did Chopin reveal in letters which parts of his work were about the singing student he was infatuated with?',
 'question: If a species may be harmed, who holds final say on whether the project may proceed?',
 'question: What country has the dog as part of its 12 animals that represent years?',
 'question: How long did his episcopate last?',
 'question: What are some of the wars the Canadian Military was involved in?',
 "question: What is worshipped during Tibet's various festibals?",
 'question: What was Apples market share of all computer shipments in 2011?',
 "question: What is central to Chopin's process?",
 'question: What year was Whitehead born?',
 'question: What percentage of plant and animal species extinction is predicted in the 21st century?',
 'question: If Bob realized he needed to mail his mother a letter after seeing the postal office, which kind of memory did he trigger?',


Tokenize data for inputs and outputs

Source: https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration

In [38]:
# Training

# Encode inputs
inputs = tokenizer(input_txt, max_length=1024, padding=True, truncation=True, return_tensors="tf")
#.input_ids
# Identify input ids and attention mask
input_ids, attention_mask = inputs.input_ids, inputs.attention_mask

# Encode outputs/targets
labels = tokenizer(output_txt, max_length=1024, padding=True, truncation=True, return_tensors="tf").input_ids

outputs = model(input_ids=input_ids, 
                attention_mask=attention_mask, 
                labels=labels)
loss = outputs.loss
logits = outputs.logits

Check an example to see what the model generates

*Note: this example context is from the training data, so not a great representation*

In [46]:
# Inference
inputs = tokenizer(
    "In Asian countries such as China, Korea, and Japan, dogs are viewed as kind protectors. The role of the dog in Chinese mythology includes a position as one of the twelve animals which cyclically represent years (the zodiacal dog).", return_tensors="tf"
)  # Batch size 1

input_ids, attention_mask = inputs.input_ids, inputs.attention_mask

outputs = model.generate(input_ids=input_ids, 
                         attention_mask=attention_mask)


print(tokenizer.decode(outputs[0], skip_special_tokens=True))




/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1695: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


the zodiacal dog. Dogs are protectors of the home. Dogs are


Next check an example the model generates when given more hyperparameters

In [45]:
# Now generate model with more sophistication (kind of)
soph_outputs = model.generate(input_ids=input_ids, 
                              attention_mask=attention_mask,
                              num_beams=2,
                              no_repeat_ngram_size=1,
                              min_length=20,
                              max_length=40)
                            

print(tokenizer.decode(soph_outputs[0], skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False))

the zodiacal dog. Dogs are viewed as protector of humans and animals in general,?


Now check an example the model generates that was not included in training data

In [47]:
inputs = tokenizer(
    "After Hurricane Katrina in 2005, Beyoncé and Rowland founded the Survivor Foundation to provide transitional housing for victims in the Houston area, to which Beyoncé contributed an initial $250,000. The foundation has since expanded to work with other charities in the city, and also provided relief following Hurricane Ike three years later.", return_tensors="tf"
)  # Batch size 1

input_ids, attention_mask = inputs.input_ids, inputs.attention_mask

outputs = model.generate(input_ids=input_ids, 
                         attention_mask=attention_mask)


print(tokenizer.decode(outputs[0], skip_special_tokens=True))


. Beyoncé and Rowland founded the Survivor Foundation in 2005.


In [49]:
# Now generate model with more sophistication (kind of)
soph_outputs = model.generate(input_ids=input_ids, 
                              attention_mask=attention_mask,
                              num_beams=2,
                              no_repeat_ngram_size=1,
                              min_length=10,
                              max_length=20)
                            

print(tokenizer.decode(soph_outputs[0], skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False))

. Beyoncé and Rowland founded the Survivor Foundation in 2006.


In [ ]:
# Find a random example to text that wasn't in training dataset
data_squad['train'][1000]

{'id': '56bfe7eaa10cfb1400551389',
 'title': 'Beyoncé',
 'context': 'After Hurricane Katrina in 2005, Beyoncé and Rowland founded the Survivor Foundation to provide transitional housing for victims in the Houston area, to which Beyoncé contributed an initial $250,000. The foundation has since expanded to work with other charities in the city, and also provided relief following Hurricane Ike three years later.',
 'question': 'How much did Beyonce initially contribute to the foundation?',
 'answers': {'text': ['$250,000'], 'answer_start': [190]}}

### Preprocess data (might want to consider later..?)

In [ ]:
# Import regular expression to help with cleaning text
# import re

In [ ]:
# def preprocess(sentence):
#     sentence=str(sentence)
#     sentence = sentence.lower()
#     sentence = sentence.replace('\n', ' ')
    
#     # Remove unicode characters
#     #sentence = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+\/\/\S+)|^rt|http.+?", "", sentence)

#     # Remove excess whitespace (if more than one space)
#     sentence =  " ".join(sentence.split())

#     return sentence

# clean_input_txt = list(map(preprocess, input_txt))

# clean_input_txt

["question_gen: context: prior to moving its headquarters to chicago, aerospace manufacturer boeing (#30) was the largest company based in seattle. its largest division is still headquartered in nearby renton, and the company has large aircraft manufacturing plants in everett and renton, so it remains the largest private employer in the seattle metropolitan area. former seattle mayor greg nickels announced a desire to spark a new economic boom driven by the biotechnology industry in 2006. major redevelopment of the south lake union neighborhood is underway, in an effort to attract new and established biotech companies to the city, joining biotech companies corixa (acquired by glaxosmithkline), immunex (now part of amgen), trubion, and zymogenetics. vulcan inc., the holding company of billionaire paul allen, is behind most of the development projects in the region. while some see the new development as an economic boon, others have criticized nickels and the seattle city council for pan